In [37]:
import pandas as pd
import numpy as np
import random
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [38]:
#Functions
def downcast_dtypes(df):
    df = df.copy()
    int_cols = df.select_dtypes('int').columns.tolist()
    float_cols = df.select_dtypes('float').columns.tolist()
    obj_cols = df.select_dtypes('object').columns.tolist()
    cat_cols = []
    for col in obj_cols:
        if col == 'pitch_type':
            continue
        if len(df[col].unique()) < len(df)/2:
            cat_cols.append(col)
      
    ints = df[int_cols].apply(pd.to_numeric,downcast='unsigned')
    floats = df[float_cols].apply(pd.to_numeric,downcast='float')
    cats = df[cat_cols].astype('category')
  
    df = df.drop(columns=int_cols + float_cols + cat_cols)
    for d in [ints, floats, cats]:
        df = pd.concat([df, d], axis=1)
    return df

def make_game_features(df):
    df = df.copy()
    
    #convert balls and strikes columns to strings (first to int)
    df['balls'] = df['balls'].astype('int').astype('str')
    df['strikes'] = df['strikes'].astype('int').astype('str')

    #create count feature:
    df['_count'] = df['balls'] + df['strikes']

    #create count category feature:
    count_map = {'00':'neutral', '21':'neutral', '32':'neutral', '10':'behind', '20':'behind',
             '30':'behind', '31':'behind', '01':'ahead', '02':'ahead', '11':'ahead',
             '12':'ahead', '22':'ahead'}
    df['count_cat'] = df['_count'].replace(count_map)
    
    #cant decide whether to drop the balls and strikes feature, keeping for now- check feature importances later
    #df = df.drop(columns=['balls', 'strikes'])
    
    #make score_differential feature
    df['score_diff'] = df['fld_score'] - df['bat_score']
    
    #make baserunners features 1/0 instead of baserunner id:
    df['on_1b_id'] = df['on_1b'] #save the runners id in case we later add feature for speedy runners
    df['on_1b'] = df['on_1b'] * 0 + 1
    df['on_1b'] = df['on_1b'].fillna(0)
    df['on_2b'] = df['on_2b'] * 0 + 1
    df['on_2b'] = df['on_2b'].fillna(0)
    df['on_3b'] = df['on_3b'] * 0 + 1
    df['on_3b'] = df['on_3b'].fillna(0)
    
    #make bases_loaded feature:
    df['bases_loaded'] = df['on_1b'] + df['on_2b'] + df['on_3b']
    df['bases_loaded'] = df['bases_loaded'].apply(lambda x: 1 if x == 3 else 0)

    return df

def gen_pitcher_percentages(df):
    df = df.copy()
    #get all the unique pitcher names in the df
    pitcher_list = df.pitcher.unique().tolist()
    #initialize empty dictionary to store each pitcher and their pitches and percentages for each pitch
    pitcher_dict = {}
    #iterate over each pitcher:
    for pitcher in pitcher_list:
        #assign the normalized value_counts to a variable
        pitch_percentages = df[df.pitcher == pitcher].pitch_type.value_counts(normalize=True)
        #convert that Series object to a dict and assign it as the value to the pitcher dictionary 
        #(pitcher name as key)
        if pitcher == 664068.0:
            pitcher_dict[664068.0] = {'FF': 1.0}
        else:
            pitcher_dict[pitcher] = pitch_percentages.to_dict()
    return pitcher_dict

def fill_pitch_type_nans(df, pitcher_dict):
    df = df.copy()
    #grab the rows where pitch_type is null:
    nulls = df[df.pitch_type.isna()]
    
    #iterate over each null row
    for index, row in nulls.iterrows():
        #use the % for that pitcher for each pitch type he throws to generate a random pitch type with 
        #that % as weight
        try:
            pitch = random.choices(population=list(pitcher_dict[row.pitcher].keys()), 
                               weights=list(pitcher_dict[row.pitcher].values()), 
                               k=1)[0]
        except KeyError:
            pvc = df[df['pitcher']==row.pitcher]['pitch_type'].value_counts(normalize=True)
            pitch_types = pvc.index.tolist()
            percentages = pvc.values.tolist()
            pitch = random.choices(population=pitch_types, weights=percentages, k=1)[0]
        except IndexError:
            print(index)
            print(row.pitcher)
                
        #fill the NaN value with the randomly generated pitch
        df.at[index, 'pitch_type'] = pitch
    return df

def make_pitch_type_cat(df):
    df = df.copy()
    
    #create map for pitch type into categories:
    pitch_type_map = {'FA':'fastball', 'FF':'fastball', 'FT':'fastball', 'FC':'fastball',
                      'FS':'fastball', 'SI':'fastball', 'SF':'fastball', 'SL':'breaking',
                      'CB':'breaking', 'CU':'breaking', 'SC':'breaking', 'KC':'breaking',
                      'CH':'offspeed', 'KN':'offspeed', 'EP':'offspeed', 'FO':'breaking', 
                      'PO':'pitchout'}

    #create pitch cateogory feature
    df['pitch_cat'] = df['pitch_type']
    df['pitch_cat'] = df['pitch_cat'].replace(pitch_type_map)
    return df

def make_strikezone_swung_and_chase_features(df):
    df = df.copy()
    
    #create swung column
    def swung(x):
        swung = ['foul','hit_into_play','swinging_strike','hit_into_play_no_out',
                 'hit_into_play_score','foul_tip','swinging_strike_blocked',
                 'foul_bunt','missed_bunt']
        return 1 if x in swung else 0
    df['batter_swung'] = df['description'].apply(swung)
    
    #initialize in_strikezone and chased features:
    df['in_strikezone'] = 1
    df['chased'] = 0
    
    df['ball_high'] = df['plate_z'] > df['sz_top']
    df['ball_low'] = df['plate_z'] < df['sz_bot']
    df['ball_left'] = df['plate_x'].apply(lambda x: x < -0.73)
    df['ball_right'] = df['plate_x'].apply(lambda x: x > 0.73)
    df['in_strikezone'] = df['ball_high'] + df['ball_low'] + df['ball_left'] + df['ball_right']
    df['in_strikezone'] = df['in_strikezone'].apply(lambda x: 0 if x > 0 else 1)
    
    nulls_subset = df[df['in_strikezone'].isna()]
    for index, row in nulls_subset.iterrows():
        if row.type == 'B':
            df.at[index, 'in_strike_zone'] = False
    
    df['chased'] = df['batter_swung'] - df['in_strikezone']
    df['chased'] = df['chased'].apply(lambda x: 1 if x == 1 else 0)
    return df

def make_batters_scouting_report(df):
    df = df.copy()
    #make list of the unique batter ids
    batters = list(df['batter'].unique())
    #initialize empty dictionary to store the batter stats
    batters_dict = {}
    #set a break flag to False for error-checking
    brk = False
    #iterate thru each unique batter
    for batter in batters:
        if brk:
            break
        #make subset of the df for that batter and assign to variable batter_df    
        batter_df = df[df['batter'] == batter]
        
        #assign all pitch categories to list:
        all_pitch_cats = ['fastball', 'breaking', 'offspeed', 'pitchout']
        #assign the pitch categories to a list
        pitch_cats = batter_df['pitch_cat'].unique().tolist()
        
        
        #get the normalized value counts of pitches by category that batter has faced
        vc = batter_df.pitch_cat.value_counts(normalize=True)
        #initialize empty dict for each batter
        batter_dict = {}
        
        #if there are any pitch categories the batter has not faced, 
        unfaced_cats = list(set(all_pitch_cats) - set(pitch_cats))
        
        #assign NaNs to his dictionary for that category
        for cat in unfaced_cats:
            if cat == 'pitchout':
                batter_dict[cat + '_perc_faced'] = 0
            else:
                batter_dict[cat + '_perc_faced'] = np.nan
                batter_dict[cat + '_chase_perc'] = np.nan
                batter_dict[cat + '_bip_swung_perc'] = np.nan
                batter_dict[cat + '_taken_strike_perc'] = np.nan
                batter_dict[cat + '_est_woba'] = np.nan
                batter_dict[cat + '_babip'] = np.nan
                batter_dict[cat + '_iso_value'] = np.nan
        
        for cat in pitch_cats:
            if brk:
                break
        
            #assign the % of pitches faced by the batter for that category to his batter dict
            
            batter_dict[cat + '_perc_faced'] = vc[cat] * 100
        
            #continue out of the loop for pitchout category since ball in play stats are NaN
            if cat == 'pitchout':
                continue
        
            #grab subset of batter df for the pitch category
            cat_df = batter_df[batter_df['pitch_cat'] == cat]
            
            #if he has faced less than 200 pitches of that type, add it to unfaced_category and fill w NaN
            if len(cat_df) < 200:
                unfaced_cats.append(cat)
                continue
        
            #calculate batters chase % for pitch type category on balls outside the strikezone
            out_of_strikezone = len(cat_df[cat_df['in_strikezone'] == 0]) #num of times ball was out of zone
            chased_count = len(cat_df[cat_df['chased'] == 1]) #num of times batter chased
            try:
                chase_perc = (chased_count / out_of_strikezone) * 100
            except ZeroDivisionError:
                chase_perc = np.nan
            #assign the chase perc to the batter dict
            batter_dict[cat + '_chase_perc'] = chase_perc
        
            #calc ball in play % for each swing for each pitch cat:
            ball_in_play_count = len(cat_df[cat_df['type'] == 'X']) #type X means ball hit into play
            swung_count = cat_df['batter_swung'].sum() #counts all the 1s in the swung column
            #assign the ball in play % per swing to the batter dict
            batter_dict[cat + '_bip_swung_perc'] = (ball_in_play_count / swung_count) * 100
        
            #calculate taken strike %
            taken_strike_count = len(cat_df[(cat_df['in_strikezone'] == 1) & (cat_df['batter_swung'] == 0)])
            pitches_in_zone_count = cat_df['in_strikezone'].sum() #counts the 1s in the in zone col
            #assign to batter_dict
            batter_dict[cat + '_taken_strike_perc'] = (taken_strike_count / pitches_in_zone_count) * 100
        
            #for each pitch type category, get the batters stats on balls hit in play
            stats = ['estimated_woba_using_speedangle', 'babip_value', 'iso_value']
            for stat in stats:
                #drop Nans from the stat column and assign to new subset, for each stat
                stat_cat_df = cat_df.dropna(subset=[stat])
                if stat == 'estimated_woba_using_speedangle':
                    #get the mean avg_est_woba
                    avg_est_woba = stat_cat_df['estimated_woba_using_speedangle'].mean()
                    #assign that value to the batters dictionary
                    batter_dict[cat + '_est_woba'] = avg_est_woba
                    if avg_est_woba == np.nan:
                        print(batter)
                        brk = True
                        break
                elif stat == 'babip_value':
                    avg_babip = stat_cat_df['babip_value'].mean()
                    batter_dict[cat + '_babip'] = avg_babip
                else:
                    avg_iso_value = stat_cat_df['iso_value'].mean()
                    batter_dict[cat + '_iso_value'] = avg_iso_value
            
        #assign the batter dictionary to the main dictionary of all batters
        batters_dict[batter] = batter_dict
    if not brk:
        print('iteration completed successfully')
        
    #make df from the batters dict
    batters_df = pd.DataFrame.from_dict(batters_dict, orient='index')
    batters_df = batters_df.reset_index().rename(columns={'index':'batter'})
    
    #merge df and batters df on batter col
    df = pd.merge(df, batters_df, how='left', on='batter')
    
    return df

def pre_process_step1(combined):
    df = combined.copy()
    #convert the pitch type for UN (unknown) to np.nan
    df['pitch_type'] = df['pitch_type'].replace({'UN':np.nan})
    #fix some faulty data that has number of balls listed as 4:
    df['balls'] = df['balls'].replace({4.0: 3.0})
    
    #count, count_cat, score_diff, on_base 1/0, bases_loaded
    df = make_game_features(df)
    
    #batter_swung, in_strikezone, chased
    df = make_strikezone_swung_and_chase_features(df)
    
    #get aggregate pitcher %s dict from prior data:
    pitcher_dict = gen_pitcher_percentages(df)
    
    #fil the NaNs for pitch_type using randomized guess from pitcher tendencies
    df = fill_pitch_type_nans(df, pitcher_dict)
        
    #pitch_type category feature
    df = make_pitch_type_cat(df)
    
    return df

In [39]:
%%time
#df14 = pd.read_pickle('pitches_2014.pkl', compression='zip')
#df15 = pd.read_pickle('pitches_2015.pkl', compression='zip')
#df16 = pd.read_pickle('pitches_2016.pkl', compression='zip')
df17 = pd.read_pickle('pitches_2017.pkl', compression='zip')
df17 = downcast_dtypes(df17)
df18 = pd.read_pickle('pitches_2018.pkl', compression='zip')
df18 = downcast_dtypes(df18)
df19 = pd.read_pickle('pitches_2019.pkl', compression='zip')

combined = pd.concat([df17, df18, df19], sort=False)

#sort and reset index
combined = combined.sort_values(by = ['game_date', 'game_pk', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

#drop a few unnecessary columns
drop_cols = ['pitcher.1','fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8','fielder_9']
combined = combined.drop(columns=drop_cols)

#iterate over every game and find the ones w/ all Nans for pitch_type and delete them from combined df
games = combined.game_pk.unique().tolist()
for game in games:
    game_df = combined[combined.game_pk == game]
    #if the game is all Nans for pitch_type
    if len(game_df.pitch_type.value_counts()) == 0:
        #delete it from combined
        combined = combined.drop(game_df.index)
        
#reset index again:
combined = combined.reset_index(drop=True)

CPU times: user 1min 1s, sys: 9.64 s, total: 1min 10s
Wall time: 1min


In [41]:
#only needed for 2016 data:

#fix some bad data for pitch_type

# pitch_types = ['FF','SL', 'FT', 'CH', 'SI', 'CU', 'FC', 'KC', 'FS', 'KN', 'IN', 'FO', 'PO','EP','SC','UN', 'FA']
# bad_data = combined[~(combined.pitch_type.isin(pitch_types))].pitch_type.value_counts().index.tolist()
# combined['pitch_type'] = combined.pitch_type.replace(to_replace=bad_data, value=np.nan)

In [42]:
#clear some memory:
import gc

#del df_16
del df17
del df18
del df19

gc.collect()

7401

In [43]:
combined.shape

(2071755, 73)

In [44]:
%%time
combined = pre_process_step1(combined)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


CPU times: user 27.3 s, sys: 3.72 s, total: 31 s
Wall time: 25.8 s


In [46]:
%%time
combined = make_batters_scouting_report(combined)

iteration completed successfully
CPU times: user 1min 4s, sys: 2.31 s, total: 1min 6s
Wall time: 1min 5s


In [47]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['team_at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['team_at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batter_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 'PH' (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batter_order_map[batter] = 'PH'      
    game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batter_order_map[x])
    return game_df

def make_batting_order_slot_feature(df):
    games = df['game_pk'].unique().tolist()
    games_list = []
    skipped_games_count = 0
    
    for game in games:
        try:
            game_df = df[df['game_pk'] == game].copy()
            pitcher_list = game_df.pitcher.unique().tolist()
        
            top = game_df[game_df['inning_topbot'] == 'Top']
            top = make_game_batting_order(top)
            bot = game_df[game_df['inning_topbot'] == 'Bot']
            bot = make_game_batting_order(bot)
            new_game_df = pd.concat([top, bot])
            new_game_df['pitcher_AB'] = new_game_df['batter'].apply(lambda x: True if x in pitcher_list else False)
            new_game_df['batting_order_slot'] = new_game_df['batting_order_slot'].where(new_game_df['pitcher_AB'] == False, other='pitcher')
            games_list.append(new_game_df)
        except:
            skipped_games_count += 1
            continue
    print('skipped games count: ' + str(skipped_games_count))
    return pd.concat(games_list).drop(columns=['pitcher_AB'])

In [48]:
%%time
combined = make_batting_order_slot_feature(combined)

skipped games count: 7
CPU times: user 7min 2s, sys: 4.45 s, total: 7min 7s
Wall time: 7min 4s


In [52]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def make_batting_order_slot_map(df):
    batting_order_slot_map = {}
    for slot in df.batting_order_slot.unique().tolist():
        batting_order_slot_map[slot] = {}
        slot_df = df[df.batting_order_slot == slot]
        
        for col in batter_cols:
            batting_order_slot_map[slot][col] = slot_df[col].mean()
    return batting_order_slot_map

def fill_batting_nans(df, batting_order_slot_map):
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))

In [53]:
%%time
batting_order_slot_map = make_batting_order_slot_map(combined)

CPU times: user 2.82 s, sys: 0 ns, total: 2.82 s
Wall time: 2.81 s


In [54]:
batting_order_slot_map

{1: {'fastball_perc_faced': 62.765084187795196,
  'fastball_chase_perc': 27.84080495082605,
  'fastball_bip_swung_perc': 40.45598656570413,
  'fastball_taken_strike_perc': 31.962136660953938,
  'fastball_est_woba': 0.37223512074559534,
  'fastball_babip': 0.2154025865714933,
  'fastball_iso_value': 0.15691701622442916,
  'breaking_perc_faced': 26.340695895944986,
  'breaking_chase_perc': 31.577836069125617,
  'breaking_bip_swung_perc': 36.89605241827341,
  'breaking_taken_strike_perc': 35.5666746896674,
  'breaking_est_woba': 0.3397664965875092,
  'breaking_babip': 0.1872640385570563,
  'breaking_iso_value': 0.13101442385072887,
  'offspeed_perc_faced': 10.883704814573973,
  'offspeed_chase_perc': 36.54483903027876,
  'offspeed_bip_swung_perc': 40.37310697509579,
  'offspeed_taken_strike_perc': 23.32780224123913,
  'offspeed_est_woba': 0.32781539532817017,
  'offspeed_babip': 0.20294449910457169,
  'offspeed_iso_value': 0.13571896819088838,
  'pitchout_perc_faced': 0.010515101697300902

In [55]:
import pickle
pickle.dump(batting_order_slot_map, open("batting_order_slot_map.pkl", "wb"))